# Setup

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
#%pip install "ultralytics<=8.3.40" supervision roboflow
import ultralytics
ultralytics.checks()

## Dataset

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets


from roboflow import Roboflow
rf = Roboflow(api_key="UmrhwHmxixp7hrYrUSIB")
project = rf.workspace("agroia").project("dataset-v1-s3fwi")
version = project.version(8)
dataset = version.download("yolov11")
                

# Train

In [ ]:
%cd {HOME}

!yolo task=segment mode=train model=yolo11s-seg.pt data=/home/nuvenpreto01/Documentos/train/yolo-11-seg/datasets/DATASET-V1-8/data.yaml epochs=40 imgsz=640 batch=4 lr0=0.001 amp=False

# Load YOLO

In [ ]:
from ultralytics import YOLO
from PIL import Image
import supervision as sv

model = YOLO(f'{HOME}/runs/segment/train/weights/best.pt')
image = Image.open("/content/datasets/Pelvis-AP-X-ray-3/test/images/45_jpg.rf.ce678a6abea6d14974aa610f0cd8c74a.jpg")
result = model.predict(image, conf=0.25)[0]
detections = sv.Detections.from_ultralytics(result)

In [ ]:
# Rename classes
name_dictionary = {
    "SUORCIL": "SOURCIL",
    "SAKRO-ILIAK-EKLEM": "SACROILIAC-JOINT",
    "ILIAK": "ILIUM"
}
detections.data["class_name"] = [name_dictionary.get(class_name, class_name) for class_name in detections.data["class_name"]]

In [ ]:
mask_annotator = sv.MaskAnnotator()
label_annotator = sv.LabelAnnotator(text_color=sv.Color.BLACK, text_position=sv.Position.CENTER)

annotated_image = image.copy()
mask_annotator.annotate(annotated_image, detections=detections)
label_annotator.annotate(annotated_image, detections=detections)

sv.plot_image(annotated_image, size=(10, 10))